In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from convnet import DiscriminatorConvNet, GeneratorConvNet
from resnet import DiscriminatorResNet, GeneratorResNet
from utils import *


### TRAINING HYPERPARAMETERS

Directly taken from paper.

In [52]:
manual_seed = 999

num_gpu = 1
lr = 1e-4
beta1 = 0
beta2 = 0.9
num_epochs = 50

dataroot="../data"
batch_size=64
workers = 4
image_size=48

real_label = 1
fake_label = -1

random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Look for available GPU(s)
device = torch.device("cuda:0" if (torch.cuda.is_available() and num_gpu > 0) else "cpu")

# Sphere-GAN CONVNET Training

In [53]:

genConv = GeneratorConvNet()
disConv = DiscriminatorConvNet()

genConv.apply(weights_init)
disConv.apply(weights_init)


DiscriminatorConvNet(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (ln1_1): LayerNorm(torch.Size([64, 48, 48]), eps=1e-05, elementwise_affine=True)
  (lrelu1_1): LeakyReLU(negative_slope=0.2, inplace)
  (conv1_2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (ln1_2): LayerNorm(torch.Size([64, 24, 24]), eps=1e-05, elementwise_affine=True)
  (lrelu1_2): LeakyReLU(negative_slope=0.2, inplace)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (ln2_1): LayerNorm(torch.Size([128, 24, 24]), eps=1e-05, elementwise_affine=True)
  (lrelu2_1): LeakyReLU(negative_slope=0.2, inplace)
  (conv2_2): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (ln2_2): LayerNorm(torch.Size([128, 12, 12]), eps=1e-05, elementwise_affine=True)
  (lrelu2_2): LeakyReLU(negative_slope=0.2, inplace)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), p

In [61]:
optimizerG = optim.Adam(genConv.parameters(), lr=lr, betas=(beta1, beta2))
optimizerD = optim.Adam(disConv.parameters(), lr=lr, betas=(beta1, beta2))

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, 128, 1, 1, device=device)
dataloader = get_stl10_dataloader(dataroot, batch_size, workers, image_size)

img_list = []
G_losses = []
D_losses = []
iters = 0
moments = 5

Files already downloaded and verified


In [77]:
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update Discriminator
        ###########################
        disConv.zero_grad()
        label = torch.full((batch_size,), real_label, device=device)
        
        # Forward pass real batch through D
        output = disConv(data[0])
        print(output.size())
        
        # Calculate loss on all-real batch
        error_disc_real = total_moment_distance(output, label, moments, device)
        
        # Calculate gradients for D in backward pass
        error_disc_real.backward()

        ## Train with all-fake batch
        
        # Generate batch of latent vectors
        noise = torch.randn(batch_size, 128, 1, 1, device=device)
        
        # Generate fake image batch with G
        fake = genConv(noise)
        label.fill_(fake_label)
        
        # Classify all fake batch with D
        output = netD(fake.detach())
        
        # Calculate D's loss on the all-fake batch
        error_disc_fake = total_moment_distance(output, label, moments, device)
        
        # Calculate the gradients for this batch
        error_disc_fake.backward()

        # Add the gradients from the all-real and all-fake batches
        error_disc = error_disc_real + error_disc_fake
        
        # Update D
        optimizerD.step()

        ############################
        # (2) Update Generator
        ###########################
        genConv.zero_grad()
        
        label.fill_(real_label)  # fake labels are real for generator cost
        
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = disConv(fake)
        
        # Calculate G's loss based on this output
        error_gen = total_moment_distance(output, label, device)
        
        # Calculate gradients for G
        error_gen.backward()
        
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(error_gen.item())
        D_losses.append(error_disc.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

torch.Size([64, 1025])


RuntimeError: Expected object of backend CPU but got backend CUDA for argument #2 'mat2'